In [13]:
!pip install albumentations
!pip install torch
!pip install torchvision
# model.py
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

'''ResNet in PyTorch.

For Pre-activation ResNet, see 'preact_resnet.py'.

Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
	Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())


# test()
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

import os
import argparse
from sklearn.metrics import classification_report

import os
import argparse

# data augment
# 訓練誤差
# lr 　optimizer 初期値　activation
# 過学習
# dataaugment validation
import cv2
import matplotlib.pyplot as plt
import albumentations as albu
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)
import numpy as np
from PIL import Image


# def load_data(data_dir):
def load_data(albu_list):
    albu_transforms = albu.Compose(albu_list)

    def albumentations_transform(image, transform=albu_transforms):
        if transform:
            image_np = np.array(image)
            augmented = transform(image=image_np)
            image = Image.fromarray(augmented['image'])
        return image

    transform_train = transforms.Compose([
        transforms.Lambda(albumentations_transform),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=128,
                                               shuffle=True, num_workers=2)

    test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=100,
                                              shuffle=False, num_workers=2)

    # train_set = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
    # train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=0)
    # test_set = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
    # test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False, num_workers=0)
    class_names = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    return train_loader, test_loader, class_names


'''Train CIFAR10 with PyTorch.'''


def main():
    dataset_name = 'CIFAR10'
    model_name = 'ResNet18'
    model_ckpt_dir = '/content/drive/My Drive/cifar10/experiment/aug/checkpoints'
    model_ckpt_path_temp = '/content/drive/My Drive/cifar10/experiment/aug/checkpoints/{}_{}_{}_epoch={}.pth'
    n_epoch = 100
    lr = 0.001

    # Make directory.
    os.makedirs(model_ckpt_dir, exist_ok=True)
    # Validate paths.
    # Set device.
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Load dataset.

    
    albu_list = [HorizontalFlip(p=0.5),
                 RandomContrast(limit=0.2, p=0.5),
                 RandomGamma(gamma_limit=(80, 120), p=0.5),
                 RandomBrightness(limit=0.2, p=0.5),
                 HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20, val_shift_limit=10, p=.9),
                 #  CLAHE(p=1.0, clip_limit=2.0),
                 ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101,
                                  p=0.8)
                 ]
    albu_list_name = [
        "HorizontalFlip",
        "RandomContrast",
        "RandomGamma",
        "RandomBrightness",
        "HueSaturationValue",
        "CLAHE",
        "ShiftScaleRotate"
    ]
    
    for albu, albuname in zip(albu_list, albu_list_name):
        train_loader, test_loader, class_names = load_data(list(albu))
    
        # Set a model.
        model = get_model(model_name)
        model = model.to(device)
    
        # Set loss function and optimization function.
        criterion = nn.CrossEntropyLoss()
        # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
        optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=5e-4, eps=1e-08, amsgrad=False)
        # Train and test.
        list_test = []
        for epoch in range(n_epoch):
            # Train and test a model.
            train_acc, train_loss = train(model, device, train_loader, criterion, optimizer)
            test_acc, test_loss = test(model, device, test_loader, criterion)
    
            # Output score.
            stdout_temp = 'epoch: {:>3}, train acc: {:<8}, train loss: {:<8}, test acc: {:<8}, test loss: {:<8}'
            print(stdout_temp.format(epoch + 1, train_acc, train_loss, test_acc, test_loss))
            # Save a model checkpoint.
            model_ckpt_path = model_ckpt_path_temp.format(dataset_name, model_name, albuname, epoch + 1)
            torch.save(model.state_dict(), model_ckpt_path)
    
            print('Saved a model checkpoint at {}'.format(model_ckpt_path))
            print('')
            list_temp = []
            list_temp = [epoch + 1, model_ckpt_path, train_acc, train_loss, test_acc, test_loss]
            list_test.append(list_temp)
        df = pd.DataFrame(list_test)
        df.columns = ["epoch", model_ckpt_path, "train_acc", "train_loss", "test_acc", "test_loss"]
        df.to_csv(f"{model_ckpt_dir}_{model_name}_{albuname}_adam_test.csv")


def train(model, device, train_loader, criterion, optimizer):
    model.train()

    output_list = []
    target_list = []
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Forward processing.
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward processing.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Set data to calculate score.
        output_list += [int(o.argmax()) for o in outputs]
        target_list += [int(t) for t in targets]
        running_loss += loss.item()

        # Calculate score at present.
        train_acc, train_loss = calc_score(output_list, target_list, running_loss, train_loader)
        if batch_idx % 10 == 0 and batch_idx != 0:
            stdout_temp = 'batch: {:>3}/{:<3}, train acc: {:<8}, train loss: {:<8}'
            print(stdout_temp.format(batch_idx, len(train_loader), train_acc, train_loss))

    # Calculate score.
    train_acc, train_loss = calc_score(output_list, target_list, running_loss, train_loader)

    return train_acc, train_loss


def test(model, device, test_loader, criterion):
    model.eval()

    output_list = []
    target_list = []
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        # Forward processing.
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Set data to calculate score.
        output_list += [int(o.argmax()) for o in outputs]
        target_list += [int(t) for t in targets]
        running_loss += loss.item()

    test_acc, test_loss = calc_score(output_list, target_list, running_loss, test_loader)

    return test_acc, test_loss


def get_model(model_name):
    if model_name == 'VGG19':
        model = VGG('VGG19')
    elif model_name == 'ResNet18':
        model = ResNet18()
    elif model_name == 'PreActResNet18':
        model = PreActResNet18()
    elif model_name == 'GoogLeNet':
        model = GoogLeNet()
    elif model_name == 'DenseNet121':
        model = DenseNet121()
    elif model_name == 'ResNeXt29_2x64d':
        model = ResNeXt29_2x64d()
    elif model_name == 'MobileNet':
        model = MobileNet()
    elif model_name == 'MobileNetV2':
        model = MobileNetV2()
    elif model_name == 'DPN92':
        model = DPN92()
    elif model_name == 'ShuffleNetG2':
        model = ShuffleNetG2()
    elif model_name == 'SENet18':
        model = SENet18()
    elif model_name == 'ShuffleNetV2':
        model = ShuffleNetV2(1)
    elif model_name == 'EfficientNetB0':
        model = EfficientNetB0()
    else:
        print('{} does NOT exist in repertory.'.format(model_name))
        sys.exit(1)

    return model


def calc_score(output_list, target_list, running_loss, data_loader):
    # Calculate accuracy.
    result = classification_report(output_list, target_list, output_dict=True)
    acc = round(result['weighted avg']['f1-score'], 6)
    loss = round(running_loss / len(data_loader.dataset), 6)

    return acc, loss

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
main()

TypeError: ignored